Преподаватель: Артур Сапрыкин
Домашнее задание.

Потренируем навыки, полученные на практическом занятии.

Пользуясь новыми знаниями, проведите эксплоративный анализ данных на данных недвижимости Бостона, и 
попытайтесь обучить линейную модель наиболее качественно. Все ваши действия и решения описывайте в markdown.

Данные вы сможете получить исполнив следующий код:
from sklearn.datasets import load_boston
import pandas as pd
data = pd.read_csv(load_boston()[‘filename’], skiprows=1)

Переменная, которую надо предсказать - ’MEDV’.

Прочитать информацию о содержимом датасета:
print(load_boston()[‘DESCR’])

Если вы неудовлетворенны результатом, но лучшего решения предложить не можете, то высылайте задание на проверку.
Цель задания - закрепить полученные навыки.

In [0]:
from sklearn.datasets import load_boston
import pandas as pd
data = pd.read_csv(load_boston()['filename'], skiprows=1)

In [3]:
print(load_boston()['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Пойдем от необходимого результата: Линейная модель + Эксплоративный анализ данных

Лучшая практика по линейной модели:
1. Эксплоративный анализ
2. Анализ полей. 
 - Категориальные переменные -> OneHotEncoder
 - Обработка выбросов (ручная + автоматическая)
*   Модель 0: Линейная регрессия (без OneHotEncoder + без Outliers и без масштабирования)
*   Модель 1: Линейная регрессия (без Outliers и без масштабирования)
*   Модель 2: Линейная регрессия (с OneHotEncoder)
*   Модель 3: Линейная регрессия (с OneHotEncoder + исключение выбросов)
*   Модель 4: Линейная регрессия (с OneHotEncoder + исключение выбросов + Исключение поля TAX (на основе ProfileReport)

3. Scaler (у=а1х1 + а2х2 + ... аnxn + w) - линейная модель чувствительна к размерностям. Хочу понять какой из skaler'ов поведет себя для улучшения. Попробую RobastScaler, который делает распределение относительно медианы и квартилей. После такого скалирования модель становится не чувствительна к выбросам
*   Модель 5: Линейная регрессия MinMaxScaler
*   Модель 6: Линейная регрессия NormalScaler
*   Модель 7: Линейная регрессия RobustScaler

4. Автоматический отбор признаков:
 - одномерные статистики (f_regression - исключает не связанные с целевой переменной признаки. Высокое p-значение указывает на то, что признак вряд ли связан с зависимой переменной)
*   Модель 8: Линейная регрессия без лишних признаков 

 - Итеративный отбор признаков (recursive feature elimination, RFE) 
 *   Модель 9: Линейная регрессия перебор признаков
*   Модель 9.1 Ленейная регрессия, но выбор признаков на основе RandomForestClassifier (автоматом)

5. Оптимизация линейной модели. В задаче написано "линейная модель" она включает в себя:
 - Линейную регрессию(простая, но нет инструментов выбора параметров. Это не удобно при большом количестве признаков и для эффективной работы с 1000ми признаков в промышленном масштабе)
  *   Модель 10: Модель 2 оптимизируем
  - Гребневая регрессия (L2 регуляризация. Некоторые Крегресии приближаются, но не равняются 0. С защитой от переобучения, ее нет у линейной регрессии + покрутим alpha - баланс между простотой модели и качеством на обучающем наборе данных)
 *   Модель 11: Гребневая регрессия

 - Лассо (L1 регуляризация.  Некоторые Крегресии приравниваются 0. Это можно рассматривать как один из видов автоматического отбора признаков. 
*   Модель 12: Регрессия Лассо

6. Попробуем RandomForestRegressor, для проверки что можно выжать из данных без упражнений с чисткой и OneHotEncoding. Сравним с результатами линейных моделей.
Погнали)
*   Модель 13: RandomForestRegressor

7. Общий вывод по оптимизации размерностей и моделей

In [0]:
# 1. Эксплоративный анализ
# Сразу сохраним копию для RandomForestRegressor и на всякий случай, если испортим данные
data_rfr = data.copy()
data_backup = data.copy()


In [5]:
data.info()
# Проверка на пропущенные значения - ОК

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null int64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null int64
TAX        506 non-null int64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
MEDV       506 non-null float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


In [6]:
data.head(5)
# Добавлю внизу чтобы на одном экране было. Разбираться с Dataset

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [7]:
data.describe()
# Разберемся какое поле, что значит

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [8]:
# Вспомогательное поле для формирования Profiling_report
import sys
!{sys.executable} -m pip install pandas-profiling

In [0]:
# Импортируем библиотеки
import pandas as pd
import numpy as np
import pandas_profiling
from pandas_profiling import ProfileReport

In [10]:
# Строим разведочный отчет по всем полям. Не забываем смотреть Toggle details
# Алгоритм оценки переменной:
# - Смотрим основные статистики (оцениваю вероятность выбросов по критерию если максимум больше чем в 10 раз 95 процентиля,
# то анализ выбросов. То же с минимумом. Если есть подозрение перехожу на вкладку Extreme values. Там Самые большие и самые маленькие значения. 
# Если идет равномерное нарастание, то нет выбросов. Если резкое возрастание - то скорее всего выброс)
# Дополнительно смотрю Distinct count, если не большое количество 
# ->Используем для определения можно ли непрерывную переменную занести 
# в категориальную и использовать OneHotEncoder
# - Гистограмму -> Определяем вид распределения -> в данном конкретном ключе не используем никак
pandas_profiling.ProfileReport(data)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,14
Number of observations,506
Total Missing (%),0.0%
Total size in memory,55.5 KiB
Average record size in memory,112.3 B
Numeric,12
Categorical,0
Boolean,1
Date,0
Text (Unique),0
Rejected,1


In [0]:
# 2. Анализ полей:
#  - Категориальные переменные -> OneHotEncoder
#       -CHAS, RAD
# Переменная TAX ProfileReport сильно коррелирована с RAD (поэтому) лучше исключить ее:
# This variable is highly correlated with RAD and should be ignored for analysis
#  Для проверки строим 2 модели: 1 с другая без переменной. Смотрим где точность выше

# Результаты анализа переменных
# Attribute Information (in order):
#         - CRIM     per capita crime rate by town - непрерывная
#         - ZN       proportion of residential land zoned for lots over 25,000 sq.ft. - непрерывная
#         - INDUS    proportion of non-retail business acres per town - непрерывная
#         - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) - категориальная (OneHotEncoder)
#         - NOX      nitric oxides concentration (parts per 10 million) - непрерывная
#         - RM       average number of rooms per dwelling - непрерывная
#         - AGE      proportion of owner-occupied units built prior to 1940  - непрерывная
#         - DIS      weighted distances to five Boston employment centres - непрерывная
#         - RAD      index of accessibility to radial highways - категориальная (OneHotEncoder)
#         - TAX      full-value property-tax rate per $10,000 - непрерывная (предварительно стоит исключить ее. Строим 2 модели: 1 с другая без переменной. Смотрим где точность выше)
#         - PTRATIO  pupil-teacher ratio by town - непрерывная
#         - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town - непрерывная
#         - LSTAT    % lower status of the population - непрерывная
#         - MEDV     Median value of owner-occupied homes in $1000's - непрерывная


#  - Обработка выбросов (ручная + автоматическая - найти способы "ленивой" обработки)
# Выбросов не нашел вручную, максимальные и минимальные значения возрастают равномерно в поле Extreme values

In [12]:
# Модель 0: Линейная регрессия (без OneHotEncoder + без Outliers и без масштабирования)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X = data.loc[:, data.columns != 'MEDV']
y = data['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
lr = LinearRegression().fit(X_train, y_train)

print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))

print("Правильность на обучающем наборе: {:.2f}".format(lr.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(lr.score(X_test, y_test)))

# Имеет место переобучение (75% на обучающей выборке и 68% на тестовой. Модель слишком оптимизирована под обучающие данные)
# Точность 68% не большая, посмотрим что получится по итогам Feature Engineering

lr.coef_: [-1.28322638e-01  2.95517751e-02  4.88590934e-02  2.77350326e+00
 -1.62388292e+01  4.36875476e+00 -9.24808158e-03 -1.40086668e+00
  2.57761243e-01 -9.95694820e-03 -9.23122944e-01  1.31854199e-02
 -5.17639519e-01]
lr.intercept_: 29.83642016383861
Правильность на обучающем наборе: 0.75
Правильность на тестовом наборе: 0.68


In [13]:
#Модель 1: Линейная регрессия (без исключения Outliers и без масштабирования)
# для переменных CHAS, RAD делаем OneHotEncoder. В Pandas это функция Get_dummies()
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

data_m1 = data.copy()

data_m1_with_dummies = pd.get_dummies( data_m1, columns = ['CHAS','RAD'] )
data_m1_with_dummies.head()


,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,B,LSTAT,MEDV,CHAS_0,CHAS_1,RAD_1,RAD_2,RAD_3,RAD_4,RAD_5,RAD_6,RAD_7,RAD_8,RAD_24
0,0.00632,18.0,2.31,0.538,6.575,65.2,4.0900,296,15.3,396.90,4.98,24.0,1,0,1,0,0,0,0,0,0,0,0
1,0.02731,0.0,7.07,0.469,6.421,78.9,4.9671,242,17.8,396.90,9.14,21.6,1,0,0,1,0,0,0,0,0,0,0
2,0.02729,0.0,7.07,0.469,7.185,61.1,4.9671,242,17.8,392.83,4.03,34.7,1,0,0,1,0,0,0,0,0,0,0
3,0.03237,0.0,2.18,0.458,6.998,45.8,6.0622,222,18.7,394.63,2.94,33.4,1,0,0,0,1,0,0,0,0,0,0
4,0.06905,0.0,2.18,0.458,7.147,54.2,6.0622,222,18.7,396.90,5.33,36.2,1,0,0,0,1,0,0,0,0,0,0


In [14]:
# И обучаем линейную регрессию снова 

X_m1 = data_m1_with_dummies.loc[:, data_m1_with_dummies.columns != 'MEDV']
y_m1 = data_m1_with_dummies['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X_m1, y_m1, random_state=42)
lr = LinearRegression().fit(X_train, y_train)

print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))

print("Правильность на обучающем наборе: {:.2f}".format(lr.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(lr.score(X_test, y_test)))

# Действие привело к улучшению обучения на тренировочных данных, но результаты на тестовых данных остались прежними. Пока действие пустое 
# + осталось переобучение как в Модели 0 

lr.coef_: [-1.30362013e-01  3.78917283e-02  6.88619778e-02 -1.72777925e+01
  4.24305248e+00 -1.06215102e-02 -1.48676397e+00 -6.10658151e-03
 -9.06026628e-01  1.34277198e-02 -5.14414107e-01 -1.28330276e+00
  1.28330276e+00 -2.69354309e+00 -1.73350724e+00  1.72202200e+00
 -1.04459253e+00 -6.54092969e-02 -2.32521228e+00  1.51259286e+00
  1.49742277e+00  3.13022681e+00]
lr.intercept_: 32.53518068681959
Правильность на обучающем наборе: 0.76
Правильность на тестовом наборе: 0.68


In [89]:
#Модель 2, 4: Линейная регрессия (с OneHotEncoder и с исключением Outliers)
# В ручную Outliers не нашел, поэтому исключаю только поле TAX как рекомендует ProfileReport
# Вернемся к Модель 2 после автоматического исключения Outliers. Пока сделаем Модель 4

data_m2 = data.copy()


data_m2_with_dummies = pd.get_dummies( data_m2, columns = ['CHAS','RAD'] )

# Исключаем поле TAX
data_m2_with_dummies = data_m2_with_dummies.loc[:, data_m2_with_dummies.columns != 'TAX']
data_m2_with_dummies.head()

X_m2 = data_m2_with_dummies.loc[:, data_m2_with_dummies.columns != 'MEDV']
y_m2 = data_m2_with_dummies['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X_m2, y_m2, random_state=42)
lr = LinearRegression().fit(X_train, y_train)

print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))

print("Правильность на обучающем наборе: {:.2f}".format(lr.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(lr.score(X_test, y_test)))

#Вывод: Исключение поля TAX не привело к улучшению модели


lr.coef_: [-1.30007038e-01  3.30336106e-02  3.37459567e-02 -1.77444854e+01
  4.25613369e+00 -1.06397909e-02 -1.48085850e+00 -9.21408730e-01
  1.35737433e-02 -5.13195903e-01 -1.34598385e+00  1.34598385e+00
 -2.40403031e+00 -1.18290790e+00  2.14929341e+00 -9.00836059e-01
  3.03007972e-02 -2.49922482e+00  1.65659651e+00  1.59582451e+00
  1.55498385e+00]
lr.intercept_: 31.197362360201808
Правильность на обучающем наборе: 0.76
Правильность на тестовом наборе: 0.68


In [0]:
# Автоматическая проверка переменных на выбросы:
#     Нормальное распределение: среднеквадратичное отклонение (σ)
#     Для всех распределений: интерквартильный интервал IQR = (75 квартиль - 25 квартиль)
#     Q1-IQR*1,5 - нижние выбросы
#     Q3+IQR*1.5 - верхние выбросы

# автоматическая проверка данных крайне важна в промышленных решениях (многие приборы самостоятельно детектируют Outliers -
# браслеты контролируюзщие сердцебиение, автомобили с самостоятельным управлением)
# +промышленные решения с 1000 столбцов (Avito, Amazon). Посмотрим что получится автоматический отбор показателей

# Источник:
#     https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba
# Так как у нас основные виды распределения далеки от нормального, используем интерквартильный интервал 

In [16]:
# В ручную выбросы не нашел. Сделаем автоматический поиск выбросов, посмотрим приведет ли это к улучшению модели
# Возьмем данные из модели 1 и рассчитает IQR для них
Q1 = data_m1.quantile(0.25)
Q3 = data_m1.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

CRIM         3.595037
ZN          12.500000
INDUS       12.910000
CHAS         0.000000
NOX          0.175000
RM           0.738000
AGE         49.050000
DIS          3.088250
RAD         20.000000
TAX        387.000000
PTRATIO      2.800000
B           20.847500
LSTAT       10.005000
MEDV         7.975000
dtype: float64


In [17]:
# напечатает выбросы. False - значит НЕ выброс. Большая таблица
print(data_m1 < (Q1 - 1.5 * IQR)) |(data_m1 > (Q3 + 1.5 * IQR))

      CRIM     ZN  INDUS   CHAS    NOX  ...    TAX  PTRATIO      B  LSTAT   MEDV
0    False  False  False  False  False  ...  False    False  False  False  False
1    False  False  False  False  False  ...  False    False  False  False  False
2    False  False  False  False  False  ...  False    False  False  False  False
3    False  False  False  False  False  ...  False    False  False  False  False
4    False  False  False  False  False  ...  False    False  False  False  False
..     ...    ...    ...    ...    ...  ...    ...      ...    ...    ...    ...
501  False  False  False  False  False  ...  False    False  False  False  False
502  False  False  False  False  False  ...  False    False  False  False  False
503  False  False  False  False  False  ...  False    False  False  False  False
504  False  False  False  False  False  ...  False    False  False  False  False
505  False  False  False  False  False  ...  False    False  False  False  False

[506 rows x 14 columns]


TypeError: ignored

In [18]:
#удаляем все автоматически найденные выбросы. Руками мы их всё равно не увидели
data_m3 = data_m1[~((data_m1 < (Q1 - 1.5 * IQR)) |(data_m1 > (Q3 + 1.5 * IQR))).any(axis=1)]
data_m3.shape

(268, 14)

In [19]:
#Для понимания сколько удалил выбросов - чуть меньше чем половину. Очень много. Потерять половину данных не лучшее решение. 
#Не беру как универсальное решение. Будем смотреть "руками"
data_m1.shape

(506, 14)

In [20]:
data_m3.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [21]:
#Модель 3: Линейная регрессия (с OneHotEncoder + исключение выбросов)
data_m3_with_dummies = pd.get_dummies( data_m3, columns = ['CHAS','RAD'] )

X_m3 = data_m3_with_dummies.loc[:, data_m3_with_dummies.columns != 'MEDV']
y_m3 = data_m3_with_dummies['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X_m3, y_m3, random_state=42)
lr = LinearRegression().fit(X_train, y_train)

print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))

print("Правильность на обучающем наборе: {:.2f}".format(lr.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(lr.score(X_test, y_test)))

#Выводы: 1. Увеличилась точность на тестовом наботе (но он стал меньше)
#2. снизилось переобучение

lr.coef_: [-1.21766754e+00 -1.53494238e-02 -1.81256303e-02  1.87230375e+00
  5.29658370e+00 -4.10105807e-02 -8.38718636e-01 -1.10663208e-02
 -7.74631376e-01  2.14513722e-02 -1.55765917e-01 -4.44089210e-16
 -5.59991313e+00 -2.20436785e+00  2.37478209e-01 -9.91691725e-01
 -7.60074850e-01 -1.76348065e+00  2.44816828e+00  1.32947536e+00
  7.30440635e+00]
lr.intercept_: 7.133204911864571
Правильность на обучающем наборе: 0.77
Правильность на тестовом наборе: 0.71


In [22]:
#Модель 4: Линейная регрессия (с OneHotEncoder + исключение выбросов + Исключение поля TAX (на основе ProfileReport)
data_m4_with_dummies = pd.get_dummies( data_m3, columns = ['CHAS','RAD'] )

# Исключаем поле TAX
data_m4_with_dummies = data_m4_with_dummies.loc[:, data_m4_with_dummies.columns != 'TAX']
data_m4_with_dummies.head()

#Модель 4:
X_m4 = data_m4_with_dummies.loc[:, data_m4_with_dummies.columns != 'MEDV']
y_m4 = data_m4_with_dummies['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X_m4, y_m4, random_state=42)
lr = LinearRegression().fit(X_train, y_train)

print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))

print("Правильность на обучающем наборе: {:.2f}".format(lr.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(lr.score(X_test, y_test)))

# Вывод: исключение TAX нам ничего не дало, только правильность на тренировочной выборке снизилась


lr.coef_: [-1.16415391e+00  4.66316273e-03 -8.94514576e-02  2.10256303e+00
  5.25716560e+00 -4.47458427e-02 -8.19000635e-01 -7.44164419e-01
  1.80154637e-02 -1.48286955e-01 -1.72084569e-15 -4.99562599e+00
 -3.68887305e-01  1.23077463e+00 -8.13612245e-01 -4.12757539e-01
 -2.24001852e+00  1.90020284e+00  1.49484748e+00  4.20507664e+00]
lr.intercept_: 4.7973496871875945
Правильность на обучающем наборе: 0.75
Правильность на тестовом наборе: 0.71


In [0]:
# 3. Scaler (у=а1х1 + а2х2 + ... аnxn + w) - модель чувствительна к размерностям. 
# Хочу понять какой из skaler'ов поведет себя для улучшения. Попробую RobastScaler, который 
# делает распределение относительно медианы и квартилей. После такого скалирования модель становится не чувствительна
# к выбросам

#Применение StandardScaler в scikit-learn гарантирует, что для каждого признака среднее будет равно 0, а дисперсия будет равна 1,
#в результате чего все признаки будут иметь один и тот же масштаб. Однако это масштабирование не гарантирует получение каких-то
#конкретных минимальных и максимальных значений признаков. 

#RobustScaler аналогичен StandardScaler в том плане, что в результате его применения признаки будут иметь один и тот же масштаб. Однако
#RobustScaler вместо среднего и дисперсии использует медиану и квартили23 . Это позволяет RobustScaler игнорировать точки данных,
#которые сильно отличаются от остальных (например, ошибки измерений)

#С другой стороны, MinMaxScaler сдвигает данные таким образом, что все признаки находились строго в диапазоне от 0 до 1. Для двумерного
#набора данных это означает, что все данные помещаются в прямоугольник, образованный осью х с диапазоном значений от 0 и 1 и
#осью у с диапазоном значений от 0 и 1.

#Normalizer осуществляет совершенно иной вид масштабирования. Он масштабирует каждую точку данных таким
#образом, чтобы вектор признаков имел евклидову длину 1. Другими словами, он проецирует точку данных на окружность с радиусом 1 (или
#сферу в случае большого числа измерений). Вектор умножается на инверсию своей длины. Подобная нормализация используется тогда,
#когда важным является направление (но не длина) вектора признаков.

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

data_m5_6_7 = data.copy()
data_m5_6_7_with_dummies = pd.get_dummies(data_m5_6_7, columns = ['CHAS','RAD'] )

#За основу возьмем самую "сделанную по лучшей практике" модель (без потери данных от автоматического обрезания) 
#и наложим на нее все виды нормализации. Посмотрим что получится 

In [24]:
#Модель 5: Линейная регрессия MinMaxScaler
# разделили dataset 
X_m5 = data_m5_6_7_with_dummies.loc[:, data_m5_6_7_with_dummies.columns != 'MEDV']
y_m5 = data_m5_6_7_with_dummies['MEDV']

# делим выборку на тест и train
X_train, X_test, y_train, y_test = train_test_split(X_m5, y_m5, random_state=42)

# масштабируем данные с помощью MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LinearRegression().fit(X_train_scaled, y_train)

print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))

print("Правильность на обучающем наборе: {:.2f}".format(lr.score(X_train_scaled, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(lr.score(X_test_scaled, y_test)))

#Вывод: MinMaxScaler не дал хорошего результата. Скорее всего данные уже Scaled
#Сравниваем с Моделью 2:
#lr.coef_: [-1.30007038e-01  3.30336106e-02  3.37459567e-02 -1.77444854e+01
  #4.25613369e+00 -1.06397909e-02 -1.48085850e+00 -9.21408730e-01
  #1.35737433e-02 -5.13195903e-01 -1.34598385e+00  1.34598385e+00
 #-2.40403031e+00 -1.18290790e+00  2.14929341e+00 -9.00836059e-01
  #3.03007972e-02 -2.49922482e+00  1.65659651e+00  1.59582451e+00
  #1.55498385e+00]
#lr.intercept_: 31.197362360201808
#Правильность на обучающем наборе: 0.76
#Правильность на тестовом наборе: 0.68



lr.coef_: [-11.59793548   3.78917283   1.82690827  -8.39700713  20.86308904
  -1.03134864 -16.3497947   -3.19984871  -8.5166503    5.32516512
 -18.64236725  -1.28330276   1.28330276  -2.69354309  -1.73350724
   1.722022    -1.04459253  -0.0654093   -2.32521228   1.51259286
   1.49742277   3.13022681]
lr.intercept_: 27.202527480321024
Правильность на обучающем наборе: 0.76
Правильность на тестовом наборе: 0.68


In [25]:
#Модель 6: Линейная регрессия StandardScaler
# разделили dataset 
X_m6 = data_m5_6_7_with_dummies.loc[:, data_m5_6_7_with_dummies.columns != 'MEDV']
y_m6 = data_m5_6_7_with_dummies['MEDV']

# делим выборку на тест и train
X_train, X_test, y_train, y_test = train_test_split(X_m6, y_m6, random_state=42)

# масштабируем данные с помощью MinMaxScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LinearRegression().fit(X_train_scaled, y_train)

print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))

print("Правильность на обучающем наборе: {:.2f}".format(lr.score(X_train_scaled, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(lr.score(X_test_scaled, y_test)))

#Вывод: StandardScaler не дал хорошего результата. Скорее всего данные уже Scaled
#Сравниваем с Моделью 2:
#lr.coef_: [-1.30007038e-01  3.30336106e-02  3.37459567e-02 -1.77444854e+01
  #4.25613369e+00 -1.06397909e-02 -1.48085850e+00 -9.21408730e-01
  #1.35737433e-02 -5.13195903e-01 -1.34598385e+00  1.34598385e+00
 #-2.40403031e+00 -1.18290790e+00  2.14929341e+00 -9.00836059e-01
  #3.03007972e-02 -2.49922482e+00  1.65659651e+00  1.59582451e+00
  #1.55498385e+00]
#lr.intercept_: 31.197362360201808
#Правильность на обучающем наборе: 0.76
#Правильность на тестовом наборе: 0.68

lr.coef_: [-1.08239674  0.87389344  0.47819282 -2.0553763   3.05132958 -0.29386083
 -3.13684016 -1.01272306 -2.04849497  1.17353356 -3.67201856 -0.34113238
  0.34113238 -0.62550667 -0.48083404  0.34686469 -0.5852767  -0.20534591
 -0.61312442  0.19937045  0.24152076  1.17582603]
lr.intercept_: 22.90791556728232
Правильность на обучающем наборе: 0.76
Правильность на тестовом наборе: 0.68


In [26]:
#Модель 7: Линейная регрессия RobustScaler
# разделили dataset 
X_m7 = data_m5_6_7_with_dummies.loc[:, data_m5_6_7_with_dummies.columns != 'MEDV']
y_m7 = data_m5_6_7_with_dummies['MEDV']

# делим выборку на тест и train
X_train, X_test, y_train, y_test = train_test_split(X_m7, y_m7, random_state=42)

# масштабируем данные с помощью RobustScaler
scaler = RobustScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LinearRegression().fit(X_train_scaled, y_train)

print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))

print("Правильность на обучающем наборе: {:.2f}".format(lr.score(X_train_scaled, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(lr.score(X_test_scaled, y_test)))

#Вывод: RobustScaler не дал хорошего результата. Скорее всего данные уже Scaled
#Сравниваем с Моделью 2:
#lr.coef_: [-1.30007038e-01  3.30336106e-02  3.37459567e-02 -1.77444854e+01
  #4.25613369e+00 -1.06397909e-02 -1.48085850e+00 -9.21408730e-01
  #1.35737433e-02 -5.13195903e-01 -1.34598385e+00  1.34598385e+00
 #-2.40403031e+00 -1.18290790e+00  2.14929341e+00 -9.00836059e-01
  #3.03007972e-02 -2.49922482e+00  1.65659651e+00  1.59582451e+00
  #1.55498385e+00]
#lr.intercept_: 31.197362360201808
#Правильность на обучающем наборе: 0.76
#Правильность на тестовом наборе: 0.68

lr.coef_: [-0.41199545  0.75783457  0.89313985 -3.2136694   3.28624414 -0.50558389
 -4.72129332 -2.36324705 -3.26169586  0.26345186 -4.90236644 -1.28330276
  1.28330276 -2.40897702 -1.44894117  2.00658807 -0.76002645  0.21915678
 -2.04064621  1.79715893  1.78198884  1.70739644]
lr.intercept_: 23.88057219467188
Правильность на обучающем наборе: 0.76
Правильность на тестовом наборе: 0.68


Общий вывод по Scaler:
В нашел DataSet применение Scaler не привело к улучшению качества прогнозирования

Автоматический отбор признаков:
одномерные статистики (f_regression - исключает не связанные с целевой переменной признаки. Высокое p-значение указывает на то, что признак вряд ли связан с зависимой переменной)

Модель 8: Линейная регрессия без лишних признаков

Итеративный отбор признаков (recursive feature elimination, RFE)

Модель 9: Линейная регрессия перебор признаков

In [56]:
#Модель 8: Линейная регрессия без признаков не связанных с целевой переменной (p-value)
from sklearn.feature_selection import SelectPercentile

data_m8 = data.copy()
data_m8_with_dummies = pd.get_dummies( data_m8, columns = ['CHAS','RAD'] )

X_m8 = data_m8_with_dummies.loc[:, data_m8_with_dummies.columns != 'MEDV']
y_m8 = data_m8_with_dummies['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X_m8, y_m8, random_state=42)

# используем f_classif (по умолчанию)
# и SelectPercentile, чтобы выбрать 50% признаков
select = SelectPercentile(percentile=50)
select.fit(X_train, y_train)

# преобразовываем обучающий набор
X_train_selected = select.transform(X_train)

print("форма массива X_train: {}".format(X_train.shape))
print("форма массива X_train_selected: {}".format(X_train_selected.shape))

форма массива X_train: (379, 22)
форма массива X_train_selected: (379, 11)


In [57]:
#Надо понять какие признаки были выбраны. Это нужно для расчета Score тестовой части выборки
mask = select.get_support()
print(mask)

# преобразовываем тестовые данные
X_test_selected = select.transform(X_test)

[ True  True  True  True  True  True False  True False False  True False
 False False  True False False False False False  True  True]


In [58]:
lr = LinearRegression().fit(X_train_selected, y_train)

print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))

lr.fit(X_train, y_train)
print("Правильность со всеми признаками: {:.3f}".format(lr.score(X_test, y_test)))
lr.fit(X_train_selected, y_train)
print("Правильность только с отобранными признаками: {:.3f}".format(
 lr.score(X_test_selected, y_test)))

#Вывод: В данном случае просто снижение количества признаков работает плохо - качество модели ухудшается 
#100% - Правильность только с отобранными признаками: 0.685
#90% - Правильность только с отобранными признаками: 0.683
#80% - Правильность только с отобранными признаками: 0.679
#70% - Правильность только с отобранными признаками: 0.673
#60% - Правильность только с отобранными признаками: 0.631
#50% - Правильность только с отобранными признаками: 0.596
#Модели явно не хватает данных для обучения. Это видно по показателю Score на тренировочных данных

lr.coef_: [-0.0955134   0.02385071  0.08454997  3.9345738   5.12382925  0.01435909
 -0.01114749 -0.60648695 -0.17968885  1.4910206   1.61152477]
lr.intercept_: -1.933525824832234
Правильность со всеми признаками: 0.685
Правильность только с отобранными признаками: 0.596


In [110]:
# Модель 9.0 Отбор признаков на основе другой модели
# То есть мы обучаем модель, в данном случае RandomForestRegressor, и из нее берем значимые признаки и подаем их в Линейную регрессию
# Это используется когда бизнес-заказчику "надо понимать" на какой бизнес-показатель воздействовать, чтобы получить результат 
# Так как Модель 13 (RandomForestRegressor) лучше подходит, попробуем применить важные признаки из нее и подать их в линейную регрессию

#Модель 9.0
data_m9 = data.copy()
data_m9_with_dummies = pd.get_dummies( data_m9, columns = ['CHAS','RAD'] )

X_m9 = data_m9_with_dummies.loc[:, data_m9_with_dummies.columns != 'MEDV']
y_m9 = data_m9_with_dummies['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X_m9, y_m9, random_state=42)

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
# Выбираем признаки на основе модели

select = SelectFromModel(RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1), threshold="median")
select.fit(X_train, y_train)
X_train_l1 = select.transform(X_train)
print("форма обуч набора X: {}".format(X_train.shape))
print("форма обуч набора X c l1: {}".format(X_train_l1.shape))

mask = select.get_support()
print(X_train.head(1))
print(mask)

X_test_l1 = select.transform(X_test)
score = LinearRegression().fit(X_train_l1, y_train).score(X_test_l1, y_test)
print("Правильность на тестовом наборе: {:.3f}".format(score))
 
 #Вывод: Подал выбранные из RandomForestRegressor параметры (половину). При этом качество линейной модели не улучшилось.
 # Не срабатывает защита от переобучения линейной модели


форма обуч набора X: (379, 22)
форма обуч набора X c l1: (379, 11)
        CRIM   ZN  INDUS    NOX     RM  ...  RAD_5  RAD_6  RAD_7  RAD_8  RAD_24
182  0.09103  0.0   2.46  0.488  7.155  ...      0      0      0      0       0

[1 rows x 22 columns]
[ True False  True  True  True  True  True  True  True  True  True False
 False False False False False False False False False  True]
Правильность на тестовом наборе: 0.660


In [120]:
#Модель 9.1: Линейная регрессия перебор признаков (выбор по результатам работы другой модели)
#метод рекурсивного исключения признаков (recursive feature elimination, RFE)
#который начинается с включения всех признаков, строит модель и
#исключает наименее важный признак с точки зрения модели. Затем
#строится новая модель с использованием всех признаков, кроме
#исключенного, и так далее, пока не останется лишь заранее определенное
#количество признаков. 

data_m9 = data.copy()
data_m9_with_dummies = pd.get_dummies( data_m9, columns = ['CHAS','RAD'] )

X_m9 = data_m9_with_dummies.loc[:, data_m9_with_dummies.columns != 'MEDV']
y_m9 = data_m9_with_dummies['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X_m9, y_m9, random_state=42)

# используем RandomForestClassifier (чтобы определить наиболее значимые признаки)
# и SelectPercentile, чтобы выбрать 50% признаков
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


select = RFE(RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1), n_features_to_select=11)
select.fit(X_train, y_train) 
mask = select.get_support()
print('Выбранные переменные:')
print(mask)
X_train_rfe= select.transform(X_train)
X_test_rfe= select.transform(X_test)
score = LinearRegression().fit(X_train_rfe, y_train).score(X_test_rfe, y_test)
print("Правильность на тестовом наборе: {:.3f}".format(score))

 #Вывод: Подал выбранные методом перебора переменные (половину). При этом качество линейной модели не улучшилось относительно.


Выбранные переменные:
[ True False  True  True  True  True  True  True  True  True  True False
 False False False False False False False False False  True]
Правильность на тестовом наборе: 0.660


In [0]:
#Оптимизация линейной модели. В задаче написано "линейная модель" она включает в себя:
#Линейную регрессию(простая, но нет инструментов выбора параметров. 
#Это не удобно при большом количестве признаков и для эффективной работы с 1000ми признаков в промышленном масштабе)
#Модель 10: Модель 2 оптимизируем

#Гребневая регрессия (L2 регуляризация. Некоторые Крегресии приближаются, но не равняются 0. 
#С защитой от переобучения, ее нет у линейной регрессии + покрутим alpha - баланс между простотой модели и качеством на обучающем наборе данных)
#Модель 11: Гребневая регрессия

#Лассо (L1 регуляризация. Некоторые Крегресии приравниваются 0. Это можно рассматривать как один из видов автоматического отбора признаков.
#Модель 12: Регрессия Лассо


In [131]:
#Модель 10: оптимизируем Линейную регрессию
# в ней нет инструментов регуляризации, и мы перебрали практически все способы улучшить модель.
#!!!!! еще возможно модели не хватает X_train для того чтобы обучиться. Давайте добавим 5-10% в обучающую выборку и посмотрим
data_m10 = data.copy()
data_m10_with_dummies = pd.get_dummies( data_m10, columns = ['CHAS','RAD'] )

#Модель 4:
X_m10 = data_m10_with_dummies.loc[:, data_m10_with_dummies.columns != 'MEDV']
y_m10 = data_m10_with_dummies['MEDV']

#Тут вместо параметра 25% на тест, устанавливаем 15% на тест и смотрим на результат:
X_train, X_test, y_train, y_test = train_test_split(X_m2, y_m2, random_state=42, test_size=0.15)
lr = LinearRegression().fit(X_train, y_train)

print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))

print("Правильность на обучающем наборе: {:.2f}".format(lr.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(lr.score(X_test, y_test)))

#Результат модели 2:
#lr.coef_: [-1.30007038e-01  3.30336106e-02  3.37459567e-02 -1.77444854e+01
  #4.25613369e+00 -1.06397909e-02 -1.48085850e+00 -9.21408730e-01
  #1.35737433e-02 -5.13195903e-01 -1.34598385e+00  1.34598385e+00
 #-2.40403031e+00 -1.18290790e+00  2.14929341e+00 -9.00836059e-01
 # 3.03007972e-02 -2.49922482e+00  1.65659651e+00  1.59582451e+00
 # 1.55498385e+00]
#lr.intercept_: 31.197362360201808
#Правильность на обучающем наборе: 0.76
#Правильность на тестовом наборе: 0.68

#Вывод: действительно, Линейной регрессии не достаточно данных, чтобы обучиться. При увеличении выборок R2 увеличивается на 9%

lr.coef_: [-1.23490399e-01  4.31098636e-02 -2.96852944e-03 -2.05428692e+01
  3.66523874e+00  3.56203649e-04 -1.67155087e+00 -9.91275304e-01
  1.13744677e-02 -5.54426171e-01 -1.26889495e+00  1.26889495e+00
 -2.59736073e+00 -1.30158977e+00  2.22167110e+00 -9.17335043e-01
 -2.22038271e-01 -2.79537527e+00  1.88581235e+00  1.41738857e+00
  2.30882707e+00]
lr.intercept_: 39.2360374844093
Правильность на обучающем наборе: 0.74
Правильность на тестовом наборе: 0.77


In [135]:
#Модель 11: Гребневая регрессия
#Регуляризации (regularization). Регуляризация означает явное ограничение модели для
#предотвращения переобучения. Регуляризация, использующаяся в гребневой регрессии, известна как L2 регуляризация.
#То есть мы регрессионные коэффициенты приближаем к 0. Это означает, что каждый признак должен иметь как можно меньшее влияние на результат (то есть
#каждый признак должен иметь небольшой регрессионный коэффициент)
data_m11_with_dummies = pd.get_dummies( data_m2, columns = ['CHAS','RAD'] )

X_m11 = data_m11_with_dummies.loc[:, data_m11_with_dummies.columns != 'MEDV']
y_m11 = data_m11_with_dummies['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X_m11, y_m11, random_state=42, test_size  = 0.15)

from sklearn.linear_model import Ridge
ridge = Ridge().fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.2f}".format(ridge.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(ridge.score(X_test, y_test)))

Правильность на обучающем наборе: 0.74
Правильность на тестовом наборе: 0.79


In [137]:
# Давайте посмотрим на скорость обучения модели Ridge
X_train, X_test, y_train, y_test = train_test_split(X_m11, y_m11, random_state=42, test_size  = 0.25)

from sklearn.linear_model import Ridge
ridge = Ridge().fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.2f}".format(ridge.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(ridge.score(X_test, y_test)))

#Вывод: Линейная модель Ridge обучается быстрее на малых выборках чем простая Линейная регрессия
# R2 растет до 0.79 (при 15% тестовых данных). 
#При этом просто Линейная регрессия R2 = 0.77 (при 15% тестовых данных) 

Правильность на обучающем наборе: 0.76
Правильность на тестовом наборе: 0.68


In [138]:
#Менее сложная модель означает меньшую правильность
#на обучающем наборе, но лучшую обобщающую способность (меньшее снижение R2 при переходе от тренировочных данных к тестовым).

from sklearn.linear_model import Ridge
ridge = Ridge(alpha=100).fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.2f}".format(ridge.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(ridge.score(X_test, y_test)))

# Вывод:
##Увеличение alpha заставляет коэффициенты сжиматься
#до близких к нулю значений, что снижает качество работы модели на
#обучающем наборе, но может улучшить ее обобщающую способность. 
# В нашем случае Правильность (R2 для линейных моделей) ухудшилась

Правильность на обучающем наборе: 0.72
Правильность на тестовом наборе: 0.66


In [139]:
#Уменьшим alpha, может получится подтянуть тестовый набор к обучающему
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1).fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.2f}".format(ridge.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(ridge.score(X_test, y_test)))
#Вывод:
# Мы вернулись к правильности классической линейной регрессии

Правильность на обучающем наборе: 0.76
Правильность на тестовом наборе: 0.68


In [140]:
#Модель 12: Регрессия Лассо
#L1 регуляризация
#Результат L1 регуляризации
#заключается в том, что при использовании лассо некоторые
#коэффициенты становятся равны точно нулю. Получается, что
#некоторые признаки полностью исключаются из модели. Это можно
#рассматривать как один из видов автоматического отбора признаков.

from sklearn.linear_model import Lasso

lasso = Lasso().fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.2f}".format(lasso.score(X_train, y_train)))
print("Правильность на контрольном наборе: {:.2f}".format(lasso.score(X_test, y_test)))
print("Количество использованных признаков: {}".format(np.sum(lasso.coef_ != 0)))
# Вывод: результаты хуже линейной регрессии на всех переменных
# Потеряли часть данных

Правильность на обучающем наборе: 0.69
Правильность на контрольном наборе: 0.63
Количество использованных признаков: 10


In [143]:
# Снижаем степень регуляризации(чтобы отобрать больше параметров)

# мы увеличиваем значение "max_iter",
# иначе модель выдаст предупреждение, что нужно увеличить max_iter.
lasso001 = Lasso(alpha=0.01, max_iter=100000).fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.2f}".format(lasso001.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(lasso001.score(X_test, y_test)))
print("Количество использованных признаков: {}".format(np.sum(lasso001.coef_ != 0)))

# Вывод: Регрессия Лоссо выродилась в Линейную регрессию

Правильность на обучающем наборе: 0.76
Правильность на тестовом наборе: 0.68
Количество использованных признаков: 20


In [152]:
# Для интереса посмотрим как Лассо будет обучаться, при увеличении объема X_train
#По сравнению с Линейной регрессией и Ridge. Ставим 15% на тест, alpha = 0.5
X_train, X_test, y_train, y_test = train_test_split(X_m11, y_m11, random_state=42, test_size  = 0.15)

lasso = Lasso(alpha=0.5, max_iter=100000).fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.2f}".format(lasso.score(X_train, y_train)))
print("Правильность на контрольном наборе: {:.2f}".format(lasso.score(X_test, y_test)))
print("Количество использованных признаков: {}".format(np.sum(lasso.coef_ != 0)))

#Вывод: Лассо при параметрах (test_size  = 0.15, alpha=0.5, max_iter=100000, Количество использованных признаков: 10 ) Правильность на контрольном наборе: 0.78
#Параметров в 2 раза меньше, чем в других линейных моделях при сопоставимом результате
#Это хорошо, с точки зрения объяснения бизнес-заказчику факторов, которые влияют на результат.

Правильность на обучающем наборе: 0.69
Правильность на контрольном наборе: 0.78
Количество использованных признаков: 10


In [90]:
# 6. Попробуем RandomForestRegressor, для проверки что можно выжать из данных без упражнений с чисткой и OneHotEncoding. Сравним с результатами линейных моделей.
#Модель 13

from sklearn.ensemble import RandomForestRegressor

#data_m13 = data.copy()
data_m13 = data_m2_with_dummies

X_m13 = data_m13.loc[:, data_m13.columns != 'MEDV']
y_m13 = data_m13['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X_m13, y_m13, random_state=42)

model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1)
model.fit(X_train, y_train)
y_test = model.predict(X_test)

print(model.feature_importances_)

#print("lr.coef_: {}".format(lr.coef_))
#print("lr.intercept_: {}".format(lr.intercept_))

print("Правильность на обучающем наборе: {:.2f}".format(model.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))

#Вывод: важен правильный подбор модели. Всё вышеперечисленное можно было не делать)


[3.94572415e-02 7.72606559e-04 4.92134541e-03 8.17112120e-03
 4.67985901e-01 1.09401198e-02 5.27184082e-02 1.60764873e-02
 5.75221300e-03 3.86958262e-01 7.02227421e-04 7.86731904e-04
 2.14652147e-04 2.49148032e-04 2.56343317e-04 2.59622621e-04
 3.68219616e-04 6.82590043e-05 9.69680404e-06 2.40123508e-04
 3.09126917e-03]
Правильность на обучающем наборе: 0.94
Правильность на тестовом наборе: 1.00


Общий вывод по работе:
1. Линейной регрессии не достаточно данных, чтобы обучиться. При увеличении количества X_train до 85% модель увеличивает R2(тест) до 77%. Это лучшее что получилось добиться в Линейной Регрессии

2. Линейная модель Ridge обучается раньше из-за регуляризации регрессоров, чем Линейная регрессия. R2 растет до 0.79 (при 15% тестовых данных). 
При этом просто Линейная регрессия R2 = 0.77 (при 15% тестовых данных)
Лассо R2 = 0.78 (test_size  = 0.15, alpha=0.5, max_iter=100000, Количество использованных признаков: 10)

3. Лучше для этих данных(небольшая выборка) подходят RandomForestRegressor.